# APA Tank Training Notebook
Completo con carga de datos, feature engineering, entrenamiento MLP y exportación.

In [ ]:

import pandas as pd
import numpy as np

# Cargar tus CSV
paths = [
    "/mnt/data/TankTraining_21_11_2025 16_54_07.csv",
    "/mnt/data/TankTraining_21_11_2025 17_00_12.csv",
    "/mnt/data/TankTraining_21_11_2025 18_10_13.csv",
    "/mnt/data/TankTraining_21_11_2025 18_13_28.csv",
]

dfs = [pd.read_csv(p) for p in paths]
data = pd.concat(dfs).reset_index(drop=True)

print("Datos cargados:", data.shape)
data.head()


In [ ]:

# Balanceo
from sklearn.utils import resample

df = data.copy()
major = df[df.action == 1]
minor = df[df.action != 1]

minor_up = resample(minor, replace=True, n_samples=len(major), random_state=42)
df_balanced = pd.concat([major, minor_up]).sample(frac=1, random_state=42).reset_index(drop=True)

print(df_balanced['action'].value_counts())


In [ ]:

# Train/Val Split + Escalado
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df_balanced.drop(columns=['time','action'], errors='ignore')
y = df_balanced['action']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,
                                                  stratify=y, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


In [ ]:

# Entrenamiento MLP
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

configs = [
    {"hidden_layer_sizes": (64,), "lr": 0.001},
    {"hidden_layer_sizes": (128,), "lr": 0.001},
    {"hidden_layer_sizes": (128, 64), "lr": 0.001},
    {"hidden_layer_sizes": (256, 128), "lr": 0.001},
]

best_acc = 0
best_model = None
best_cfg = None

for cfg in configs:
    mlp = MLPClassifier(
        hidden_layer_sizes=cfg["hidden_layer_sizes"],
        activation="relu",
        solver="adam",
        learning_rate_init=cfg["lr"],
        max_iter=2500,
        random_state=42
    )
    mlp.fit(X_train, y_train)
    acc = accuracy_score(y_val, mlp.predict(X_val))
    print(cfg, "=>", acc)
    if acc > best_acc:
        best_acc = acc
        best_model = mlp
        best_cfg = cfg

print("BEST:", best_cfg, best_acc)


In [ ]:

# Métricas finales
from sklearn.metrics import confusion_matrix, classification_report

pred = best_model.predict(X_val)
print(confusion_matrix(y_val, pred))
print(classification_report(y_val, pred))


In [ ]:

# Exportación
from Utils import ExportAllformatsMLPSKlearn, WriteStandardScaler

X_for_onnx = np.array(X_train[:1], dtype=np.float32)

ExportAllformatsMLPSKlearn(
    mlp=best_model,
    X=X_for_onnx,
    picklefileName="mlp_tank.pkl",
    onixFileName="mlp_tank.onnx",
    jsonFileName="mlp_tank.json",
    customFileName="mlp_tank_custom.txt"
)

WriteStandardScaler(
    "scaler_values.txt",
    scaler.mean_,
    scaler.var_
)

print("Exportación completada.")
